##IPerl Notebook implementation of the Gaussian Network Model with a flexible coarse-grain model of the protein

Demian Riccardi, March 6, 2015

####Description
This notebook implements the [Gaussian Network Model (GNM) of Bahar, Atilgan, and Erman](http://www.sciencedirect.com/science/article/pii/S1359027897000242) using HackaMol and the Perl Data Language (PDL) and compares the calculated fluctuations to the crystallographic B-Factors. This notebook will be extended to calculate the atom-atom correlations. GNM is a significant simplification of the [Elastic Network Model introduced by Tirion](http://journals.aps.org/prl/abstract/10.1103/PhysRevLett.77.1905), which was already a drastic simplification of protein dynamics.  This notebook uses HackaMol, PDL, and the iPerl kernel, written by Zaki Mughal, of the iPython notebook.

In [1]:
use Modern::Perl;
use HackaMol;
use Time::HiRes qw(time);

####1. Download molecule from Protein DataBank and read it into a HackaMol::Molecule object:

In [2]:
my $bldr = HackaMol->new;

my $pdb = "2cba.pdb";
$bldr->getstore_pdbid($pdb); #download from pdb and store it locally (to reduce multiple downloads)

my $mol = HackaMol->new
                  ->read_file_mol($pdb); # object methods chain from left to right


2cba.pdb exists, set self->overwrite(1) to overwrite at reply input line 4.
you can load this file using something like HackaMol->new->read_file_mol at reply input line 4.
HackaMol::Molecule=HASH(0x7fb52bf7a5d8)


####2. Coarse-grain the molecule
GNM coarse-grains the C$_\alpha$ atoms of the protein backbone. Let's pull out those with full occupancy and create a new HackaMol::Molecule object:

In [3]:
my $t1 = time;
my @resids = sort {$a->get_atoms(0)->resid <=> $b->get_atoms(0)->resid  } 
                   HackaMol->new->group_by_atom_attr( 'resid', $mol->all_atoms);
my $rcut = 3.0;

my $mol_cg = HackaMol::AtomGroup->new();

foreach my $res ( @resids) {

    my @cas =  grep {$_->occ == 1.0} grep {$_->name eq 'CA'}  $res->all_atoms;
    next unless @cas;
    
    $mol_cg->push_atoms( @cas );     
    my $sidechain  =  HackaMol::AtomGroup->new(atoms=> [
                                                    grep {$_->name ne 'CA'}
                                                    grep {$_->name ne 'O'}
                                                    grep {$_->name ne 'N'}
                                                    grep {$_->name ne 'C'} 
                                                    grep {$_->Z != 1 } 
                                                    grep {$_->occ == 1.0} $res->all_atoms
                                                ]
    );
    next unless $sidechain->count_atoms;

    my $scat_com = HackaMol::Atom->new(
                             Z=>54, 
                             resname     => $cas[0]->resname,
                             resid       => $cas[0]->resid,
                             name        => 'XE',
                             record_name => $cas[0]->record_name,
                             bfact       => $cas[0]->bfact,
                             coords      => [$sidechain->COM],
    );

    $mol_cg->push_atoms($scat_com)  if($cas[0]->distance($scat_com) >= $rcut);

}

my $dt = time - $t1;

printf ("Time coarse-graining: %3.3fs\n",$dt);


Time coarse-graining: 0.050s


1


####3. Next, calculate a Kirchoff matrix using a single parameter: the cutoff distance
The Kirchoff matrix (K) (also referred to as the connectivity matrix) is simple to implement. It is a square matrix with each dimension being the number of atoms, i.e. K(1:N,1:N). The cutoff distance is the parameter that determines whether or not two atoms are connected.  If the cutoff distance is small, the matrix is sparse (most elements are zero). The elements of matrix are evaluated as follows:

$ K = \left\{ 
\begin{array}{l l}
  -1                 &                                                  \quad \mbox{if $i\ne j$ and $R_{ij} \le R_{cut} $}\\
                   0 &                                                  \quad \mbox{if $R_{ij} > R_{cut} $}\\ 
 - \displaystyle \sum_{i,j\ne i} K_{ij} & \quad  \mbox{if $i = j$}  
 \end{array} \right. $

Below, we define the cutoff distance, square it (to avoid calculating the square root), and then loop over the atomic coordinates to construct the matrix.      

In [4]:
sub kirchoff_crunch{
    #args: AtomGroup (or Molecule) Cutoff distance
    my $mol   = shift;
    my $rcut  = shift;
    
    my $rsqr  = $rcut*$rcut;
    my $N     = $mol->count_atoms;
    my @xyzs  = map{ $_->xyz } $mol->all_atoms ;

    my @K;    # Kirchoff matrix
#    my @dist; # keep a matrix of the distances

    foreach my $i (0 .. $#xyzs){
        my $xyz_i = $xyzs[$i];
        foreach my $j ($i+1 .. $#xyzs){
            my $d2 = $xyzs[$j]->dist2($xyz_i);
 #           $dist[$i][$j] = $d2;
            my $dxyz2 = $xyzs[$j]->dist2($xyz_i);
            if ($dxyz2 <= $rsqr){
                $K[$i][$j]--;
                $K[$j][$i]--;
                $K[$i][$i]++;
                $K[$j][$j]++;
            }
        }
    }

    return (\@K);
    
}

####4. Compute the pseudo-inverse of the Kirchoff matrix and compare the B-factors.
Here, we use PDL and a PDL interface to Lapack (PDL::LinearAlgebra).  Since the  first eigenvalue of the Kirchoff matrix is zero, we calculate the pseudoinverse (using the mpinv function). 

In [5]:
use PDL::Lite;
use PDL::LinearAlgebra qw(mpinv);
use PDL::Stats::Basic;
use Time::HiRes qw(time);

my $bfact_exp = pdl(map {$_->bfact} grep {$_->name eq 'CA'} $mol_cg->all_atoms); 
my @ica = grep {$mol_cg->get_atoms($_)->name eq 'CA'} 0 .. $mol_cg->count_atoms -1;

my $tgnm1 = time;

my $K = kirchoff_crunch($mol_cg,7.5);

my $kirch_pdl  = pdl(@{$K});
my $pinv_kirch = $kirch_pdl->mpinv;

my $tgnm2 = time;

printf ("time: %.3f\n", $tgnm2-$tgnm1, 0 ); 
my $bfact_calc = pdl(@{$pinv_kirch->diag->unpdl}[@ica]);
printf ("Pearson coefficient: %.3f\n",  $bfact_calc->corr($bfact_exp));

time: 0.079
Pearson coefficient: 0.786


1


While there are limitations of comparisons between B-factors and theoretical models that will not be discussed here, a correlation above 0.7 is generally pretty good.  Next scale the calculated fluctuations so that the averages match and then print them out for plotting:

In [6]:
$bfact_calc = $bfact_exp->avg*$bfact_calc/$bfact_calc->avg;
printf("%10.3f %10.3f\n",$bfact_exp->at($_), $bfact_calc->at($_) ) foreach (0 .. $bfact_exp->nelem-1);

    21.110     27.934
    13.740     15.939
    10.510     11.060
    11.880     10.624
    18.310     18.874
    22.880     19.468
    19.110     21.940
    13.760     12.033
    11.930     11.346
    11.030     10.943
    10.720     16.054
     9.330     13.271
     9.580     10.298
    10.770     11.611
    11.630     17.315
    13.370     18.788
    12.320     12.592
    12.090     14.383
    11.140     13.572
    10.100     11.278
    13.480     13.091
    14.240     13.494
    14.010     13.387
     8.980      8.865
     5.780      7.270
     5.390      7.354
     5.430      6.830
     5.400      9.084
     7.530      9.734
     9.820     10.444
    12.390     15.143
    13.540     15.072
    17.240     26.426
    19.110     22.226
    15.530      9.965
    16.840     12.132
    18.230     11.047
    22.170     12.421
    28.470     20.223
    28.640     23.952
    25.540     13.658
    24.310     15.159
    19.910     15.972
    16.290     12.251
    14.870     13.668
    16.080